In [1]:
import torch
import numpy as np
import torchvision

In [ ]:
import os
from shutil import copyfile

In [ ]:
class DataSplitter:
    def __init__(self, part):
        self.part = part

    def split(self, folder, train, test):
        filenames = os.listdir(folder)
        #print(filenames)
        for (i, fname) in enumerate(filenames):
            src = os.path.join(folder, fname)
            if (i % self.part) == 0:
                dst = os.path.join(test, fname)
            else:
                dst = os.path.join(train, fname)
            #print(src, dst)
            copyfile(src, dst)

In [ ]:
# 10% of the data will be used for validation
ds = DataSplitter(10)
ds.split(r"D:\Python Projects\AgePredictor\UTKFace", r"D:\Python Projects\AgePredictor\Training", r"D:\Python Projects\AgePredictor\Testing")

In [ ]:
# split training into folders
dir = r"D:\Python Projects\AgePredictor\Training"+"\\"
filenames = os.listdir(dir)
for file in filenames:
    
    age = int(file.split("_")[0])
    full = dir+file
    
    if age < 11:
        os.rename(full,r"D:\Python Projects\AgePredictor\train\0to10\\"+file)
    elif age < 21:
        os.rename(full,r"D:\Python Projects\AgePredictor\train\\11to20\\"+file)
    elif age < 31:
        os.rename(full,r"D:\Python Projects\AgePredictor\train\\21to30\\"+file)
    elif age < 41:
        os.rename(full,r"D:\Python Projects\AgePredictor\train\\31to40\\"+file)
    elif age < 61:
        os.rename(full,r"D:\Python Projects\AgePredictor\train\\41to60\\"+file)
    else:
        os.rename(full,r"D:\Python Projects\AgePredictor\train\\61plus\\"+file)


In [ ]:
# split test into folders
dir = r"D:\Python Projects\AgePredictor\Testing"+"\\"
filenames = os.listdir(dir)
for file in filenames:
    
    age = int(file.split("_")[0])
    full = dir+file
    
    if age < 11:
        os.rename(full,r"D:\Python Projects\AgePredictor\test\0to10\\"+file)
    elif age < 21:
        os.rename(full,r"D:\Python Projects\AgePredictor\test\\11to20\\"+file)
    elif age < 31:
        os.rename(full,r"D:\Python Projects\AgePredictor\test\\21to30\\"+file)
    elif age < 41:
        os.rename(full,r"D:\Python Projects\AgePredictor\test\\31to40\\"+file)
    elif age < 61:
        os.rename(full,r"D:\Python Projects\AgePredictor\test\\41to60\\"+file)
    else:
        os.rename(full,r"D:\Python Projects\AgePredictor\test\\61plus\\"+file)


In [16]:
# Pytorch model creation
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder

In [17]:
#train and test data directory
train_dir = r"D:\Python Projects\AgePredictor\train"
test_dir = r"D:\Python Projects\AgePredictor\test"

In [18]:
#load the train and test data
dataset = ImageFolder(train_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))
test_dataset = ImageFolder(test_dir,transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

In [19]:
img, label = dataset[0]
print(img.shape,label)

torch.Size([3, 150, 150]) 0


In [20]:
dataset.classes

['0to10', '11to20', '21to30', '31to40', '41to60', '61plus']

In [21]:
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split

In [22]:
batch_size = 128
val_size = 2000
train_size = len(dataset) - val_size

train_data,val_data = random_split(dataset,[train_size,val_size])
print(f"Length of Train Data : {len(train_data)}")
print(f"Length of Validation Data : {len(val_data)}")

Length of Train Data : 19277
Length of Validation Data : 2000


In [23]:
train_dl = DataLoader(train_data, batch_size, shuffle = True, num_workers = 4, pin_memory = True)
val_dl = DataLoader(val_data, batch_size*2, num_workers = 4, pin_memory = True)

In [24]:
# make a class that extends the functionality of torch.nn.Module

import torch.nn as nn
import torch.nn.functional as F



In [25]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [26]:

class AgeClassification(ImageClassificationBase):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            
            nn.Conv2d(3, 32, kernel_size = 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(32,64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
        
            nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128 ,128, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256,256, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            
            nn.Flatten(),
            nn.Linear(82944,1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512,6)
        )
    
    def forward(self, xb):
        return self.network(xb)

In [27]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

  
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

  
def fit(epochs, lr, model, train_loader, val_loader, opt_func = torch.optim.SGD):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        model.epoch_end(epoch, result)
        history.append(result)
    
    return history

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [28]:
num_epochs = 1
opt_func = torch.optim.Adam
lr = 0.001
model = AgeClassification().to(device)
#fitting the model on training data and record the result after each epoch
history = fit(num_epochs, lr, model, train_dl, val_dl, opt_func)

Epoch [0], train_loss: 1.4423, val_loss: 1.2181, val_acc: 0.5091
